#Pre-request


## **Mount** Google Drive
# ===============================


In [ ]:
from google.colab import drive
drive.mount('/content/drive')




## Source the project from get hub
# ===============================


In [ ]:

#!ls /content/drive/MyDrive/Sem-6/coding/github/fraud_detection
project_path = "/content/drive/MyDrive/Sem-6/coding/github/fraud_detection/"
%cd $project_path
#!pip install -r requirements.txt



/content/drive/MyDrive/Sem-6/coding/github/fraud_detection
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 161.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 122.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 136.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 171.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 137.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 169.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 155.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 174.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.6/111.6 kB 296.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 111.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9

##Install requirement

In [ ]:
%pip install -r requirements.txt --upgrade --no-cache-dir --force-reinstall


#import

In [ ]:
%pip freeze > requirements-lock.txt

import pandas as pd
import yaml
import logging
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from ydata_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np


#Helper Code

##Loging

In [ ]:
# Make sure results directory exists
os.makedirs("results", exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler("results/data_prep.log")
    ]
)
logger = logging.getLogger(__name__)



##Load config

In [ ]:
def load_config(config_path="configs/baseline.yaml"):
    """Load YAML config file"""
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    logger.info(f"Loaded config from {config_path}")
    return config

##Load  CDRs

In [ ]:
def load_cdr(file_path, nrows=None):
    """Load a CDR CSV file with optional row limit (sample mode)."""
    logger.info(f"Loading file: {file_path} with nrows={nrows}")
    return pd.read_csv(file_path, nrows=nrows)


def load_all_data(config):
    """
    Load all CSVs defined in config into a dict of DataFrames.
    Uses training.sample_size if available.
    """
    base = config["dataset"]["base_path"]
    files = config["dataset"]["files"]
    sample_size = config.get("training", {}).get("sample_size", None)

    data = {}
    for name, fname in files.items():
        path = os.path.join(base, fname)
        df = load_cdr(path, nrows=sample_size)
        data[name] = df
        logger.info(f"Loaded {name} -> {df.shape} from {path}")
    return data


##Build Features

In [ ]:

def build_feature_table(data: dict):
    """
    Merge APP, SMS, USER, VOC datasets on phone_no_m into a single feature table.
    Each dataset is aggregated per phone_no_m, then merged into one DataFrame.

    Parameters
    ----------
    data : dict
        Dictionary of raw DataFrames, e.g. {"app": df_app, "sms": df_sms, "user": df_user, "voc": df_voc}

    Returns
    -------
    feature_df : pd.DataFrame
        Merged feature table with one row per phone_no_m
    """



    feature_parts = []

    # --------------------------
    # APP features
    # --------------------------
    if "app" in data:
        df = data["app"].copy()
        app_feat = df.groupby("phone_no_m").agg(
            app_count=("busi_name", "nunique"),
            total_flow=("flow", "sum"),
            avg_flow=("flow", "mean")
        ).reset_index()
        feature_parts.append(app_feat)

    # --------------------------
    # SMS features
    # --------------------------
    if "sms" in data:
        df = data["sms"].copy()
        sms_feat = df.groupby("phone_no_m").agg(
            sms_count=("opposite_no_m", "count"),
            unique_contacts=("opposite_no_m", "nunique")
        ).reset_index()
        feature_parts.append(sms_feat)

    # --------------------------
    # USER features (static profile)
    # --------------------------
    if "user" in data:
        df = data["user"].copy()
        user_feat = df.drop_duplicates(subset=["phone_no_m"])
        feature_parts.append(user_feat)

    # --------------------------
    # VOC features (calls)
    # --------------------------
    if "voc" in data:
        df = data["voc"].copy()
        voc_feat = df.groupby("phone_no_m").agg(
            call_count=("opposite_no_m", "count"),
            unique_callers=("opposite_no_m", "nunique"),
            avg_call_dur=("call_dur", "mean"),
            total_call_dur=("call_dur", "sum")
        ).reset_index()
        feature_parts.append(voc_feat)

    # --------------------------
    # Merge all features
    # --------------------------
    from functools import reduce
    feature_df = reduce(
        lambda left, right: pd.merge(left, right, on="phone_no_m", how="outer"),
        feature_parts
    )

    # --------------------------
    # Handle missing values
    # --------------------------
    feature_df = feature_df.fillna(0)

    return feature_df


##Genrate profile

In [ ]:
def generate_profile(df, output_file="results/profile.html", mode="minimal"):
    """
    Generate profiling report (ydata-profiling) with safe defaults.

    mode:
        - "minimal": disables heavy stats, safe for big or messy data
        - "full": full profiling
    """
    # Drop very high-cardinality or long-text columns in minimal mode
    if mode == "minimal":
        drop_cols = [
            col for col in df.columns
            if df[col].nunique() > 500 or df[col].astype(str).str.len().mean() > 50
        ]
        if drop_cols:
            print(f"⚠️ Skipping columns for profiling: {drop_cols}")
            df = df.drop(columns=drop_cols)

    profile = ProfileReport(
        df,
        title="Fraud Detection EDA Report",
        explorative=True,
        plot={"wordcloud": False},
        correlations={ "pearson": {"calculate": mode=="full"},
                      "spearman": {"calculate": mode=="full"},
                      "kendall": {"calculate": mode=="full"},
                      "phi_k": {"calculate": mode=="full"},
                      "cramers": {"calculate": mode=="full"},
                      "auto": {"calculate": mode=="full"}},
        interactions={"continuous": mode=="full"},
        missing_diagrams={"heatmap": mode=="full"},
        duplicates={"calculate": mode=="full"}   # ✅ keep only supported params
    )

    profile.to_file(output_file)
    print(f"✅ Profiling report saved → {output_file}")
    return output_file


##Preprocess_features

In [ ]:
def preprocess_features(df, numeric_cols, categorical_cols):
    """
    Scale numeric features and one-hot encode categorical features.
    Ensures categorical columns are converted to string type.
    Compatible with sklearn <1.2 and >=1.2.
    """
    # Scale numeric
    scaler = StandardScaler()
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

    # Ensure categorical cols are all strings
    df[categorical_cols] = df[categorical_cols].astype(str)

    # OneHotEncoder compatibility
    try:
        encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")  # sklearn >=1.2
    except TypeError:
        encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")         # sklearn <1.2

    encoded = encoder.fit_transform(df[categorical_cols])
    encoded_df = pd.DataFrame(
        encoded,
        columns=encoder.get_feature_names_out(categorical_cols),
        index=df.index
    )

    # Combine
    df = pd.concat([df.drop(columns=categorical_cols), encoded_df], axis=1)
    return df

#


# Load config +  dataset
# ===============================


In [ ]:
config = load_config("configs/baseline.yaml")

# Load raw data
data = load_all_data(config)

# Build feature table
feature_df = build_feature_table(data)

# Generate professional profile
# Generate profiles per raw dataset
profiling_mode = config["training"].get("profiling_mode", "minimal")

for name, df in data.items():
    output_file = f"results/profile_{name}.html"
    generate_profile(df, output_file=output_file, mode=profiling_mode)
